## Simulation-Based Inference: Type1a Supernovae
## A Simple Phantom Energy Model
Created: Mar. 21, 2023, CLASHEP 2023, San Esteban, Chile, Harrison B. Prosper<br>
Updated: Mar. 28, 2023, HBP

### Cosmological Model
The cosmological model used in the simulation-based tutorial (__phantom.ipynb__) is defined by the dimensionless energy density 
\begin{equation}
    \Omega(a) = \exp(a^n - 1) \, / \, a^3,
\end{equation}

where the index $n$ is a free parameter. This model can be integrated exactly, yielding 

\begin{align}
    H_0 t & = \int_0^a \frac{dy}{y \sqrt{\Omega(y)}}, \\
          & = \sqrt{e} \, 2^{\frac{3}{2 n}}  \gamma\left(\frac{3}{2 n}, \frac{a^n}{2}\right) \, / \, n , 
\end{align}

where $\gamma(a, x) = \int_0^x \, t^{a - 1} \, e^{-t} \, dt$ is the 
[lower incomplete gamma function](https://en.wikipedia.org/wiki/Incomplete_gamma_function). The inverse function

\begin{equation}
    a(t)  = \sqrt[n]{2 \gamma^{-1}(3/(2n), \, n 2^{-3/(2n)} H_0 t \, / \sqrt{e})}
\end{equation}

gives the dimensionless scale factor $a$ as a function of time $t$ since the Big Bang. The scale factor is defined so that it is equal to one today, which defines the age of the universe, 

\begin{align}
    H_0 t_0 
& = \sqrt{e} \, 2^{\frac{3}{2 n}}  \gamma\left(\frac{3}{2 n}, \frac{1}{2}\right) \, / \, n .
\end{align}

Interestingly, the dependence of the scale factor $a(t)$ on  the time $t$ is practically indistinguishable from that of the standard $\Lambda$-CDM model for $a < 1$. But the model exhibits 
a future singularity (a __Big Rip__) characterized by the condition $a \rightarrow \infty$ at a *finite* time. In our model, the Big Rip occurs at

\begin{equation}
    H_0 t_\textrm{rip} = \sqrt{e} \, 2^{\frac{3}{2n}} \, \Gamma\left(\frac{3}{2n}\right) \, / \, n,
\end{equation}

that is, at 

\begin{align}
    t_\textrm{rip} & = \frac{\Gamma\left(\frac{3}{2n}\right)}{\gamma\left(\frac{3}{2 n}, \frac{1}{2}\right) } \, t_0 .
\end{align}

The quantity
\begin{equation}
    \mu(z, \theta) = 5 \log_{10}[(1 + z) \, \sin(\sqrt{-\Omega_K} \, u(z, \theta))\, / \, \sqrt{-\Omega_K}] - 5 \log_{10}(H_0) + 5 \log_{10}(c) + 25,
\end{equation}
where
\begin{equation}
    u(z, \theta) \equiv \int_{1/(1+z)}^{1} \frac{da}{a^2\sqrt{\Omega(a)}} ,
\end{equation}

is the theoretical prediction for the __distance modulus__, an astronomical measure of distance. $\Omega_K$ is the curvature parameter, which we set to zero; that is, we assume a model universe in which the global geometry of space, defined by hypersurfaces with $t = \textrm{constant}$, is flat. With $\Omega_K = 0$, our cosmological model is defined by only two parameters, namely, $n$ and and the Hubble constant $H_0$. The distance modulus simplifies to 

\begin{align}
    \mu(z, \theta) 
    & = 5 \log_{10}\left[ (1 + z) c \, u(z, \theta) \, /\,  H_0 \right] + 25 .
\end{align}

For this cosmological model, the dimensionless quantity $u(z, \theta)$ is given by

\begin{align}
u(z, \theta) = \frac{2^{\frac{1}{2 n}} \left(\gamma\left(\frac{1}{2 n}, \frac{1}{2}\right) - \gamma\left(\frac{1}{2 n}, \frac{\left(\frac{1}{z + 1}\right)^{n}}{2}\right)\right) e^{\frac{1}{2}}}{n} ,
\end{align}

where, again, $\gamma(a, x)$ is the lower incomplete gamma function. For a flat universe, $\Omega_K = 0$, $d_L = c (1+z) u(z, \theta) / H_0$ is the __luminosity distance__, which is defined so that the relationship between the energy flux $f$ received by an observer and the luminosity $L$ of the source has the same form as that in a non-expanding universe, namely, $f = L \, / \, (4\pi d_L^2)$.

This notebook defines several cosmological functions including the __distance_modulus($z$, $n$, $H_0$)__.

In [24]:
import os, sys

# the standard module for array manipulation
import numpy as np

# standard scientific python module
import scipy as sp
import scipy.stats as st
import scipy.optimize as op

# standard plotting module
import matplotlib as mp
import matplotlib.pyplot as plt

# make plots appear inline
%matplotlib inline

# update fonts
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 18
        }
mp.rc('font', **font)
mp.rc('xtick', labelsize='x-small')
mp.rc('ytick', labelsize='x-small')

# set usetex = False if Latex is not available on your system
mp.rc('text', usetex=True)

### Distance modulus
  1. $u(z, \theta)$ dimensionless luminosity distance, $d_L$, which for a flat universe is  related to $u(z, \theta)$ as follows $d_L = c \, (1+z) u(z, \theta) \, / \, H_0$.
  1. $\mu(z, \theta)$ distance modulus for a flat universe.

\begin{align}
    \mu(z, \theta) & = 5 \log_{10}[(1 + z) \, \sin(\sqrt{-\Omega_K} \, u(z, \theta))\, / \, \sqrt{-\Omega_K}] - 5 \log_{10}(H_0) + 5 \log_{10}(c) + 25, \\
    & = 5 \log_{10}\left[ (1 + z) c \, u(z, \theta) \, /\,  H_0 \right] + 25,
\end{align}

setting $\Omega_K = 0$.

In [25]:
def u(z, n):
    q = 2*n
    b = 1/q
    x = 0.5/(1+z)**n
    A = sp.special.gamma(b)
    A = A**q
    A = np.exp(0.5) * (2*A)**b
    y = A * (sp.special.gammainc(b, 0.5) - sp.special.gammainc(b, x))
    return y/n

class DistanceModulus:
    def __init__(self):
        self.offset = 5*np.log10(2.99e5) + 25
        
    def __call__(self, z, n, H0):
        return 5*np.log10((1+z)*u(z, n)/H0) + self.offset

distance_modulus = DistanceModulus()

### Lifetime of model universe

\begin{align}
H_0 t_0 & = \sqrt{e} \, 2^{\frac{3}{2 n}}  \gamma\left(\frac{3}{2 n}, \frac{1}{2}\right) \, / \, n .
\end{align}

In [15]:
def lifetime(n):
    b = 3/2/n
    return np.sqrt(np.e) * 2**b * sp.special.gammainc(b, 0.5) * sp.special.gamma(b) / n

### Big rip

\begin{equation}
    H_0 t_\textrm{rip} = \sqrt{e} \, 2^{\frac{3}{2n}} \, \Gamma\left(\frac{3}{2n}\right) \, / \, n,
\end{equation}

In [16]:
def big_rip_time(n):
    b = 3/2/n
    return np.sqrt(np.e) * 2**b * sp.special.gamma(b) / n

### Scale factor
\begin{equation}
    a(t)  = \sqrt[n]{2 \gamma^{-1}(3/(2n), \, n 2^{-3/(2n)} H_0 t \, / \sqrt{e})}
\end{equation}

In [17]:
class ScaleFactor:
    def __init__(self):
        self.roote = np.sqrt(np.e)
        
    def __call__(self, t, n):
        b = 3/2/n
        x = t * n / 2**b / self.roote / sp.special.gamma(b)
        y = 2 * sp.special.gammaincinv(b, x)
        m = 1/n
        return y**m
    
scale_factor = ScaleFactor()

### Read data from the Union2.1 file

In [18]:
def read_data(filename):
    # skip first 5 rows then read columns 1, 2, and 3
    Z, X, dX = np.loadtxt(filename, 
                            delimiter='\t', 
                            skiprows=5, 
                            usecols=(1,2,3), 
                            unpack=True)

    print("number of observations: %d" % len(Z))
    print("%5s\t%10s\t%10s +/- %-10s" % ('', 'z', 'x', 'dx'))
    for ii, (z, x, dx) in enumerate(zip(Z, X, dX)):
        if ii % 100 == 0:
            print("%5d\t%10.3f\t%10.4f +/- %-10.4f"% (ii, z, x, dx))
            
    return (Z, X, dX)

### Define negative log-likelihood

In [19]:
def nll(pars, *args):
    n, H0 = pars[0], pars[1]
    bag   = args[0]
    
    f = distance_modulus(bag.z, n, H0)     
    c = (bag.x - f) / bag.dx
    c = c * c
    return c.sum()/2

### Plot Type1a data

In [20]:
class Scribe:
    def __init__(self, xpos, line, nlines=12, ftsize=14):
        
        self.ftsize = ftsize
        
        axes = plt.gca()
        self.xmin, self.xmax = axes.get_xlim()
        self.ymin, self.ymax = axes.get_ylim()
        self.ystep = (self.ymax-self.ymin) / nlines
        self.xpos  = self.xmin + xpos * (self.xmax - self.xmin)  
        self.ypos  = self.ymax - self.ystep * line
        
    def __def__(self):
        pass
    
    def write(self, text, skip=0, indent=0, color='black'):
        offset = indent*(self.xmax-self.xmin)
        self.ypos = self.ypos - skip*self.ystep
        
        plt.text(self.xpos+offset, self.ypos, text, 
                 fontsize=self.ftsize, 
                 color=color)
        
        self.ypos -= self.ystep

In [23]:
# define ranges for redshifts and distance moduli
ZMIN  =  0.0 
ZMAX  =  1.5
MUMIN = 32.0
MUMAX = 46.0

def plot_data(bag, nll=None,
              zmin=ZMIN, zmax=ZMAX, 
              mumin= MUMIN, mumax=MUMAX, 
              ftsize=12, 
              fgsize=(5, 4)):
  
    # set size of figure
    plt.figure(figsize=fgsize)
    
    plt.errorbar(bag.z, bag.x, yerr=bag.dx, 
                 fmt='o', 
                 ecolor='steelblue', markersize=1,
                 color='black', label='data')
        
    # set up x, y limits
    plt.xlim(zmin, zmax)
    plt.ylim(mumin, mumax)
   
    # add x and y labels
    plt.xlabel('$z$', fontsize=16)
    plt.ylabel('$x$', fontsize=16)
    
    # annotate 
    xpos   = 0.3 # as a fraction of x-range  
    scribe = Scribe(xpos, line=6, ftsize=ftsize)
    scribe.write('The Union2.1 Compilation')     
    scribe.write('The Supernova Cosmology Project')
       
    if nll:
        filename = bag.name + '_union_fit.png'
        
        # res:   fitted parameters
        # p:     parameters to be passed to distanceModulus
        chi2 = 2 * nll(bag.res, bag)
                
        ndf  = len(bag.z)-len(bag.res) #number of degrees of freedom
    
        # compute best-fit model
        nz   = 100
        zstep= (zmax - zmin) / nz
        z    = np.arange(zmin+zstep/2, zmax, zstep)
        n    = bag.res[0]
        H0   = bag.res[1]
        mu   = distance_modulus(z, n, H0)

        plt.plot(z, mu, color='red', label=f'{bag.name:s} model')
        
        scribe.write(r"$\chi^{2} / {\rm ndf} = %5.1f / %d = %5.2f$"%\
                     (chi2, ndf, chi2/ndf))
        
        plt.legend(fontsize=14)
    else:
        filename = "type1a_union_2_1_data.png"
    
    # tighten layout so that image is fully
    # contained within viewport
    
    plt.tight_layout()
    
    print('\n%s' % filename)
    plt.savefig(filename)
    plt.show()